# Image Classification

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import os
import tensorflow as tf
import numpy as np
 
SEED = 1234
tf.random.set_seed(SEED)  

cwd = os.getcwd()

In [ ]:
dataset_dir = os.path.join(cwd, 'MaskDataset')
test_dir = os.path.join(dataset_dir, 'test')
valid_dir = os.path.join(dataset_dir, 'validation')
train_dir = os.path.join(dataset_dir, 'training')

#### Let's create generators to read images from dataset directory
Here we can decide whether to apply data augmentation or not

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True
# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(horizontal_flip=True,
                                        brightness_range=[0.7,1.3],
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)
    
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
def get_data_from_directory(dataset_directory):
    return os.listdir(dataset_directory)

In [ ]:
training_set = get_data_from_directory(train_dir)
validation_set = get_data_from_directory(valid_dir)

#### We create two dataframes for both training and validation set
- We create two dictionaries containing the labels of the classes each image belongs to
- Then we use these dictionaries to create two Dataframe objects that will help us to link together each image with the corresponding label 

In [ ]:
import json, operator
import pandas as pd
from collections import OrderedDict

with open(os.path.join(dataset_dir, "train_gt.json")) as f:
    train_dic = json.load(f)

# Let's split the dictionary into two different ones, one for training and one for validation
valid_dic = {}
for data in validation_set:
    valid_dic[data] = train_dic.get(data)
    del train_dic[data]
    
# Training dataframe
dataframe_train = pd.DataFrame(train_dic.items())
dataframe_train.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe_train["class"] = dataframe_train["class"].astype(str)

# Validation dataframe
dataframe_valid = pd.DataFrame(valid_dic.items())
dataframe_valid.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe_valid["class"] = dataframe_valid["class"].astype(str)

- Through the method `flow_from_dataframe()` we create our final generator, that will contain the rescaled images

In [ ]:
#img size
img_h = 224
img_w = 224

#batches
bs = 24

#Training
train_gen = train_data_gen.flow_from_dataframe(dataframe_train,
                                               train_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

#Validation
valid_gen = valid_data_gen.flow_from_dataframe(dataframe_valid,
                                               valid_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

Found 4207 validated image filenames belonging to 3 classes.
Found 1407 validated image filenames belonging to 3 classes.


N.B.: The formula to calculate the __maximum batch__ size is the following: 
$$ Max Batch Size
= \frac{Available GPU memory Bytes}{4*(Size Of Tensors + Trainable Parameters)} ,
$$              
where: *size of tensors = img_h x img_w x #channels* , and *#channels = 3* in case of __RGB__  
Source: https://stackoverflow.com/questions/46654424/how-to-calculate-optimal-batch-size

#### Create Dataset objects
These objects are needed to perform the `fit()` function on the model later on

In [ ]:
num_classes = 3

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
# Without calling the repeat function the dataset will be empty after consuming all the images (it'll block at first epoch)
train_dataset = train_dataset.repeat()

# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

## Build the model
1. import the VGG19 CNN
2. decide wheter to perform transfer learning or fine-tuning 
3. attach to the CNN two other convolutional blocks
4. build the classifier

In [ ]:
cnn = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [ ]:
cnn.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
transfer_learning = False

if transfer_learning:
    freeze_until = 25
    for layer in cnn.layers[:freeze_until]:
        layer.trainable = False
else:
    for layer in cnn.layers:
        layer.trainable = True

model = tf.keras.Sequential()
model.add(cnn)

for j in range(2):
    for i in range(4):
        model.add(tf.keras.layers.Conv2D(filters=1024, 
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(units=512))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(units=256))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 1024)        4719616   
_________________________________________________________________
batch_normalization (BatchNo (None, 7, 7, 1024)        4096      
_________________________________________________________________
re_lu (ReLU)                 (None, 7, 7, 1024)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 1024)        9438208   
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 1024)        4096      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 7, 7, 1024)        0

## Reload the weights

The purpose of the following cell is to continue the training from pre-saved weights.

**Follow these steps before running the code**:
1. download the zip file at https://drive.google.com/file/d/10MWFSJdQqXtZgBrYsaExmDBvFSRPMCQL/view?usp=sharing
2. unzip the downloaded file in the current working directory
3. run the cell

In [ ]:
def get_model_path(model_name, epoch_number):
    path = os.path.join(cwd, model_name, 'ckpts')
    for file in os.listdir(path):
        if str(epoch_number) in file:
            path = os.path.join(path, "cp_" + str(epoch_number) + ".ckpt")
            return path
        
model_name = 'Model_54_Nov20_08-36-19'
epochs = 28                                           # <--- modify this

path = get_model_path(model_name, epochs)

model.load_weights(path)

## Compile the model

### - define optimization parameters


In [ ]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy() # because we have one-hot encoded labels (as specified by the 
                                                 # class_mode parameter in flow_from_dataframe())

# Learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

### - save model/weights
Folders are created following an incremental index that wille uniquely name the model together with the timestamp recording the time it has been created.

In [ ]:
import os
from datetime import datetime

cwd = os.getcwd()

exps_dir = os.path.join('tensorboard_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)
    model_name = "01"
else:
    # gets the name of the last model and generates the new name
    model_name = np.array(os.listdir(exps_dir))[-1]
    model_name = model_name[6:8]
    if int(model_name) in range(9):
        model_name = "Model_0" + str(int(model_name)+1)
    else:
        model_name = "Model_" + str(int(model_name)+1)

now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

### - create checkpoint

In [ ]:
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)
callbacks.append(ckpt_callback)

### - prepare tensorboard log

In [ ]:
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

### - apply early stopping

In [ ]:
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    callbacks.append(es_callback)

## Train the model

In [ ]:
model.fit(x=train_dataset,
          epochs=100,  # set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Train for 176 steps, validate for 59 steps
Epoch 1/100
176/176 [==============================] - 110s 626ms/step - loss: 0.0693 - accuracy: 0.9788 - val_loss: 0.9040 - val_accuracy: 0.8195
Epoch 2/100
176/176 [==============================] - 83s 471ms/step - loss: 0.1728 - accuracy: 0.9418 - val_loss: 0.7246 - val_accuracy: 0.8522
Epoch 3/100
176/176 [==============================] - 82s 469ms/step - loss: 0.0755 - accuracy: 0.9755 - val_loss: 0.4217 - val_accuracy: 0.8721
Epoch 4/100
176/176 [==============================] - 83s 472ms/step - loss: 0.0572 - accuracy: 0.9845 - val_loss: 0.3612 - val_accuracy: 0.9026
Epoch 5/100
176/176 [==============================] - 83s 473ms/step - loss: 0.0597 - accuracy: 0.9810 - val_loss: 0.4510 - val_accuracy: 0.8955
Epoch 6/100
176/176 [==============================] - 82s 467ms/step - loss: 0.0570 - accuracy: 0.9812 - val_loss: 0.4085 - val_accuracy: 0.8742
Epoch 7/100
176/176 [==============================] - 82s 468ms/step - loss: 0.

## Test the model
### - create a generator for the test set

In [ ]:
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             target_size=(img_h, img_w), 
                                             color_mode='rgb',
                                             batch_size=bs, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 450 images belonging to 1 classes.


### - make predictions
We have these classes:
    - 0: "NO PERSON in the image is wearing a mask"
    - 1: "ALL THE PEOPLE in the image are wearing a mask"
    - 2: "SOMEONE in the image is not wearing a mask"

In [ ]:
import ntpath

predictions = model.predict(test_gen, verbose=1)

results = {}

images = test_gen.filenames
i = 0

for p in predictions:
    prediction = np.argmax(p)
    image_name = ntpath.basename(images[i])
    results[image_name] = str(prediction)
    i += 1

19/19 [==============================] - 8s 436ms/step


## Create the prediction csv file

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
create_csv(results, exp_dir)